# NDC Analytics App

## Main

This notebook contains an ETL pipeline for NDC data from UNFCCC's [NDC Registry](https://unfccc.int/NDCREG) for the NDC Analytics App. It is largely self-contained and shares only two functions with the core codebase: one for embedding texts and another one for obtaining a database connection for LanceDB.

### Libraries

In [1]:
import os
import re
from datetime import datetime
from pathlib import Path
from typing import Literal

import geopandas as gpd
import pandas as pd
import requests
import tiktoken
from bs4 import BeautifulSoup, Tag
from dotenv import load_dotenv
from pydantic import AnyUrl, BaseModel, Field, computed_field
from tqdm import tqdm

from src.genai import embed_texts
from src.database import get_connection

# load the env variables
load_dotenv()

True

In [2]:
# run version in the format YY-MM-DD defines file/filder names to use
VERSION = "25-06-04"

# for optimal performance, the tokeniser must match the model used in the app
tokeniser = tiktoken.encoding_for_model("gpt-4o-mini")
# create the directories if necessary
if not (PATH_DATA := Path("data")).exists():
    print(f"Created {PATH_DATA} directory.")
    PATH_DATA.mkdir()
if not (CORPUS_DATA := Path("corpus", VERSION)).exists():
    print(f"Created {CORPUS_DATA} directory.")
    CORPUS_DATA.mkdir()

### Classes

Data models that define core entities in the pipeline.

In [3]:
class Page(BaseModel):
    """
    Extracted page from a PDF document.
    """

    index: int = Field(
        description="Page index in the source file (0-based numbering).", ge=0
    )
    text: str = Field(description="Text extracted from the page.")


class Chunk(BaseModel):
    """
    Chunk from a PDF document that may span multiple pages.
    """

    pages: tuple[int, int] = Field(
        description="Start and end pages a text chunk is extracted from (0-based numbering)"
    )
    text: str = Field(
        description="Text chunk based on source text tokenisation. It does not correspond to any semantic unit per se, e.g., paragraph, sentence or idea."
    )


class Document(BaseModel):
    """
    Basic document model with a title and URL.
    """

    title: str = Field(description="Title of the document.")
    url: AnyUrl = Field(description="URL of the document file, e.g., PDF or Word.")

    @classmethod
    def from_tag(cls, a: Tag) -> "Document":
        """
        Create a Document instance from an HTML tag.

        Parameters
        ----------
        a : Tag
            HTML anchor tag to create a Document from.

        Returns
        -------
        Document
            Parsed Document from the HTML tag.
        """
        return cls(title=re.sub(r"\s+", " ", a.text).strip(), url=a.get("href"))


class NDC(BaseModel):
    """
    NDC data model for entries in the NDC Registry.
    """

    party: str = Field(description="Party who submitted the NDC.")
    original: Document = Field(description="Original NDC Document object.")
    translation: Document | None = Field(
        description="Translation of the NDC Document object if available."
    )
    languages: list[str] = Field(description="Languages of languages used in the NDC.")
    version: int = Field(description="Numeric version of the NDC.")
    status: Literal["Active"] = Field(
        description="Status of the NDC. Included for completeness"
    )
    date: datetime = Field(
        description="Date the NDC was published as shown in the Registry."
    )

    @classmethod
    def from_tag(cls, tr: Tag) -> "NDC":
        """
        Create an NDC instance from an HTML tag.

        Parameters
        ----------
        tr : Tag
            HTML tr tag to create an NDC from.

        Returns
        -------
        NDC
            Parsed NDC from the HTML tag.
        """
        party, original, languages, translation, version, status, date, *_ = (
            tr.find_all("td")
        )
        # parse the original
        a = original.find("a", class_=re.compile("is-original"))
        original = Document.from_tag(a)
        # parse the translation
        if (a := translation.find("a", class_=re.compile("is-translation"))) is None:
            translation = None
        else:
            translation = Document.from_tag(a=a)
        # parse the date
        date = datetime.strptime(date.text.strip(), "%d/%m/%Y")
        return cls(
            party=party.text.strip(),
            original=original,
            translation=translation,
            languages=languages.text.strip().split("\n"),
            version=span.text if (span := version.find("span")) is not None else 0,
            status=status.text.strip(),
            date=date,
        )

    @computed_field
    @property
    def document(self) -> Document:
        """
        Computed property to get the translated document or, if not available,
        the original document.

        Returns
        -------
        Document
            Translated document, if available, otherwise original document.
        """
        return self.translation or self.original

### Functions

Functions for extracting, transformning and loading the data.

In [ ]:
def download_file(url: str, file_path: str, session: requests.Session) -> str:
    """
    Download a file from a URL.

    Parameters
    ----------
    url : str
        URL to a file.
    file_path : str
        Path to file on disk.
    session : requests.Session
        Requests session object to reuse the connection.

    Returns
    -------
    str
        Path to downloaded file.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        " AppleWebKit/537.36 (KHTML, like Gecko)"
        " Chrome/102.0.0.0 Safari/537.36"
    }
    with session.get(url, headers=headers, stream=True, timeout=30) as response:
        response.raise_for_status()
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    return file_path


def extract_text(file_path: str, session: requests.Session) -> tuple[str, list[Page]]:
    """
    Extract text from a file using AIaaS API from DFx (https://dfx-aiaas-api.azurewebsites.net).

    This function uses an internal API service at UNDP. The service can extract text from PDF
    and Word documents. You can modify this function to perform text extraction and langiage
    identification in any other way as long as its signature is unchanged.

    Parameters
    ----------
    file_path : str
        Path to file on disk. Only PDF and docx files are supported.
    session : requests.Session
        Requests session object to reuse the connection.

    Returns
    -------
    tuple[str, list[Page]]
        A tuple of size 2 containing a ISO 639-3 code for the identified language,
        e.g., "eng", "esp", "fra", and a list of pages, each with page index and
        extracted text.
    """
    with open(file_path, "rb") as file:
        response = session.post(
            url="https://dfx-aiaas-api.azurewebsites.net/extraction",
            params={"clean": True, "language": True},
            headers={"api_key": os.environ["AIAAS_API_KEY"]},
            files={"file": file},
            timeout=30,
        )
    data = response.json()
    language = data["language"]["code"]
    pages = [Page(**page) for page in data["pages"]]
    return language, pages


def get_text_length(text: str) -> int:
    """
    Calculate text length in terms of tokens using OpenAI tokeniser, ignoring
    non-alphanumeric artefacts.

    Parameters
    ----------
    text : str
        Input text.

    Returns
    -------
    int
        Number of tokens in the text.
    """
    # strip punctuation, numbers and special characters
    text = "".join(filter(lambda x: x.isalpha() or x.isspace(), text))
    tokens = tokeniser.encode(text)
    return len(tokens)


def get_pages_length(pages: list[Page] | None) -> float:
    """
    Calculate average page length in terms of tokens using OpenAI tokeniser.

    Parameters
    ----------
    pages : list[Page] | None
        List of pages as returned by `extract_text` if available.

    Returns
    -------
    float
        Average page length in terms of tokens.
    """
    if pages is None:
        return 0.0
    return sum(get_text_length(page.text) for page in pages) / len(pages)


def chunk_pages(pages: list[Page], size: int = 256) -> list[dict]:
    """
    Chunk pages into smaller chunks of specified size using OpenAI tokeniser.

    Parameters
    ----------
    pages : list[Page]
        List of pages as returned by `extract_text`.
    size : int, default=256
        Optional chunk size in terms of tokens.

    Returns
    -------
    chunks : list[dict]
        List of chunks containing text and page indices.
    """
    # collect all tokens and each token's page index
    tokens, indices = [], []
    for page in pages:
        tokens_ = tokeniser.encode(page.text + " ")
        indices_ = [page.index] * len(tokens_)
        tokens.extend(tokens_)
        indices.extend(indices_)

    # chunk texts using tokens and get the corresponding page indices
    chunks = []
    overlap = size // 2  # ~50% overlap
    for i in range(0, len(tokens), overlap):
        text = tokeniser.decode(tokens[i : i + size]).strip()
        pages = sorted(set(indices[i : i + size]))
        if not text:
            continue
        chunks.append({"text": text, "pages": pages})
    return chunks


def load_data(df: pd.DataFrame) -> int:
    """
    Load data into an "ndcs" table in LanceDB, overwriting all data
    already in the table.

    The function also creates a full-text search index on `text`
    column as well as scalar indices on other columns

    Parameters
    ----------
    df : pd.DataFrame
        Data frame containing NDC data.

    Returns
    -------
    int
        Number of rows uploaded to the database.
    """
    db = get_connection()
    table = db.create_table("ndcs", data=df, mode="overwrite")
    table.create_fts_index("text", use_tantivy=False)
    indices = [
        ("language", "BITMAP"),
        ("date", "BTREE"),
        ("iso", "BITMAP"),
        ("categories", "LABEL_LIST"),
        ("version", "BITMAP"),
    ]
    for column, index_type in indices:
        table.create_scalar_index(column, index_type=index_type)
    return table.count_rows()

### Data Collection

- Collect data from the NDC registry.
- Add metadata from UNSD M49.
- Prepare geospetial boundaries data from UN Geospatial.

#### NDC Metadata

Use this section to parse metadata from the NDC Registry and download the files.

1. Go to [NDC Registry](https://unfccc.int/NDCREG).
2. Manually save the HTML table tag (or the whole page) to `ndc-registry-v{YY-MM-DD}.html` in `data` directory, where `YY-MM-DD` matches the value set above, e.g., `ndc-registry-v25-06-02.html`.
3. Extract metadata from the HTML file using the code in this section.

In [5]:
with open(PATH_DATA.joinpath(f"ndc-registry-v{VERSION}.html")) as file:
    soup = BeautifulSoup(file)
table = soup.find("table")
rows = table.find_all("tr", class_=re.compile("^submission-nid-.+"))
print("Rows:", len(rows))

Rows: 217


In [6]:
df_registry = pd.DataFrame(NDC.from_tag(row).model_dump() for row in rows)
print("Shape:", df_registry.shape)
display(df_registry.head())

Shape: (217, 8)


,party,original,translation,languages,version,status,date,document
0,Nepal,{'title': 'Nationally Determined Contribution ...,None,[English],3,Active,2025-05-19,{'title': 'Nationally Determined Contribution ...
1,Republic of Moldova,"{'title': 'Republic of Moldova NDC 3.0', 'url'...",None,[English],3,Active,2025-05-06,"{'title': 'Republic of Moldova NDC 3.0', 'url'..."
2,Kenya,{'title': 'Kenya's Second Nationally Determine...,None,[English],0,Active,2025-04-30,{'title': 'Kenya's Second Nationally Determine...
3,Zambia,"{'title': 'Provisional NDC 3.0', 'url': https:...",None,[English],0,Active,2025-03-10,"{'title': 'Provisional NDC 3.0', 'url': https:..."
4,Cuba,"{'title': 'Cuba NDC 3.0', 'url': https://unfcc...",None,[Spanish],0,Active,2025-02-27,"{'title': 'Cuba NDC 3.0', 'url': https://unfcc..."


#### Geographical Areas Metadata

Use this section to add ISO codes to NDC data. The codes are needed for displaying results on a map.

1. Got to [Standard country or area codes for statistical use (M49)](https://unstats.un.org/unsd/methodology/m49/).
2. Manually download the classification (go to Full view under Search and download and download a CSV file).
3. Add ISO country codes to NDC metadata.

In [7]:
df_m49 = pd.read_csv(PATH_DATA.joinpath("UNSD — Methodology.csv"), sep=";")
print("Shape:", df_m49.shape)
display(df_m49.head())

Shape: (248, 15)


,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [8]:
# convert to dict and adjust some names to match `party` column
mapping = dict(df_m49[["Country or Area", "ISO-alpha3 Code"]].values)
mapping["Côte d'Ivoire"] = "CIV"
mapping["European Union (EU)"] = "EU"  # dummy iso
mapping["Netherlands"] = "NLD"

In [9]:
df_registry["iso"] = df_registry["party"].map(mapping)
mask = df_registry["iso"].isna()
assert not mask.any(), "Add mapping for the missing parties: {}".format(
    df_registry.loc[mask, "party"].tolist()
)

print("Shape:", df_registry.shape)
display(df_registry.head())

Shape: (217, 9)


,party,original,translation,languages,version,status,date,document,iso
0,Nepal,{'title': 'Nationally Determined Contribution ...,None,[English],3,Active,2025-05-19,{'title': 'Nationally Determined Contribution ...,NPL
1,Republic of Moldova,"{'title': 'Republic of Moldova NDC 3.0', 'url'...",None,[English],3,Active,2025-05-06,"{'title': 'Republic of Moldova NDC 3.0', 'url'...",MDA
2,Kenya,{'title': 'Kenya's Second Nationally Determine...,None,[English],0,Active,2025-04-30,{'title': 'Kenya's Second Nationally Determine...,KEN
3,Zambia,"{'title': 'Provisional NDC 3.0', 'url': https:...",None,[English],0,Active,2025-03-10,"{'title': 'Provisional NDC 3.0', 'url': https:...",ZMB
4,Cuba,"{'title': 'Cuba NDC 3.0', 'url': https://unfcc...",None,[Spanish],0,Active,2025-02-27,"{'title': 'Cuba NDC 3.0', 'url': https://unfcc...",CUB


In [10]:
# reshape into a long format, i.e., every row represents a file now, as some parties have original and translated NDCs
df_registry = df_registry.melt(
    id_vars=["iso", "party", "version", "date"],
    value_vars=["original", "translation"],
    var_name="type",
    ignore_index=True,
)
df_registry.dropna(subset=["value"], ignore_index=True, inplace=True)
df_registry = df_registry.join(pd.DataFrame(df_registry["value"].tolist()))
df_registry.drop("value", axis=1, inplace=True)
print("Shape:", df_registry.shape)
display(df_registry.head())

Shape: (233, 7)


,iso,party,version,date,type,title,url
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...
1,MDA,Republic of Moldova,3,2025-05-06,original,Republic of Moldova NDC 3.0,https://unfccc.int/sites/default/files/2025-05...
2,KEN,Kenya,0,2025-04-30,original,Kenya's Second Nationally Determined Contribut...,https://unfccc.int/sites/default/files/2025-05...
3,ZMB,Zambia,0,2025-03-10,original,Provisional NDC 3.0,https://unfccc.int/sites/default/files/2025-03...
4,CUB,Cuba,0,2025-02-27,original,Cuba NDC 3.0,https://unfccc.int/sites/default/files/2025-02...


In [11]:
# download the files
with requests.Session() as session:
    for index, row in tqdm(df_registry.iterrows(), total=len(df_registry)):
        file_extension = "docx" if str(row.url).endswith(".docx") else "pdf"
        file_name = f"{row.iso.lower()}-{row.type}-{row.date:%y-%m-%d}.{file_extension}"
        file_path = CORPUS_DATA.joinpath(file_name)
        try:
            download_file(row.url, file_path, session)
        except Exception as e:
            # manually download documents that have triggered an error
            print(e, row.url, file_name)
        finally:
            df_registry.at[index, "file_name"] = file_name

print("Shape:", df_registry.shape)
display(df_registry.head())

 38%|███▊      | 89/233 [01:22<12:53,  5.37s/it]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')) https://unfccc.int/sites/default/files/2022-06/NDC%20-%20Guatemala%202021.pdf gtm-original-22-05-23.pdf


100%|██████████| 233/233 [02:23<00:00,  1.62it/s]

Shape: (233, 8)


,iso,party,version,date,type,title,url,file_name
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf
1,MDA,Republic of Moldova,3,2025-05-06,original,Republic of Moldova NDC 3.0,https://unfccc.int/sites/default/files/2025-05...,mda-original-25-05-06.pdf
2,KEN,Kenya,0,2025-04-30,original,Kenya's Second Nationally Determined Contribut...,https://unfccc.int/sites/default/files/2025-05...,ken-original-25-04-30.pdf
3,ZMB,Zambia,0,2025-03-10,original,Provisional NDC 3.0,https://unfccc.int/sites/default/files/2025-03...,zmb-original-25-03-10.pdf
4,CUB,Cuba,0,2025-02-27,original,Cuba NDC 3.0,https://unfccc.int/sites/default/files/2025-02...,cub-original-25-02-27.pdf


- Manually redownload the documents that are corrupted or stored as `.docx`.
- Note that there is the same NDC document (submitted by Spain) for all EU member states.

In [12]:
df_registry.to_csv(PATH_DATA.joinpath(f"ndc-registry-v{VERSION}.csv"), index=False)

#### Geospatial Data

Use this section to prepare geospatial data for displaying results on a map.


1. Manually downloaded the official UN area boundaries from [UN Geoportal on ArcGIS](https://geoportal.un.org/arcgis/home/item.html?id=d7caaff3ef4b4f7c82689b7c4694ad92) (newer versions might exist).
2. The **file does not work** with Plotly out of the box and **requires transformation into gj2008** specification. See [this comment on GitHub](https://github.com/mbloch/mapshaper/issues/432#issuecomment-675775465) for more details.
3. Read the gj2008-transformed GeoJSON of area polygons from the United Nations Secretariat.
4. Merge area boundaries for all subareas within a country code. For example, there are three rows for `PRT` (Portugal) because the mainland country, Madeira Island and Azores Islands are treated as three separate shapes. The merge might not be perfect, see a disclaimer in the app.
5. Save the file as a package artifact in `src/data` to be used by the app.

In [13]:
# read the official boundaries
df_bnda = gpd.read_file(PATH_DATA.joinpath("bnda-simplified-gj2008.geojson"))
print("Shape:", df_bnda.shape)
display(df_bnda.head())

Shape: (262, 19)


,objectid,iso3cd,m49_cd,nam_en,lbl_en,georeg,geo_cd,sub_cd,int_cd,subreg,intreg,iso2cd,lbl_fr,name_fr,globalid,stscod,admiso,globalid_1,geometry
0,1,ABW,533,Aruba,Aruba (Neth.),AME,19.0,419.0,29.0,Latin America and the Caribbean,Caribbean,AW,Aruba (Pays-Bas),Aruba,{5491D09D-7C2C-419A-87AD-9C3E005C3402},4,NLD,{50AEFBBE-1DEE-4D77-86FB-D45BD05A3870},"POLYGON ((-70.05101 12.62242, -70.04847 12.621..."
1,2,AFG,004,Afghanistan,AFGHANISTAN,ASI,142.0,34.0,NaN,Southern Asia,None,AF,AFGHANISTAN,Afghanistan,{7DE6B67F-2269-4D5A-AFB4-EE86B273DD16},1,AFG,{ECA2DF74-59A0-49DB-B4F4-1C6D61C2E8CD},"POLYGON ((74.88986 37.23409, 74.69366 37.2662,..."
2,3,AGO,024,Angola,ANGOLA,AFR,2.0,202.0,17.0,Sub-Saharan Africa,Middle Africa,AO,ANGOLA,Angola,{E0A579DF-0564-4CA0-8835-4698965F1432},1,AGO,{DDA48D4F-C6C1-4C06-9317-D980B8C27856},"MULTIPOLYGON (((23.99925 -10.89024, 24.03344 -..."
3,4,AIA,660,Anguilla,Anguilla *,AME,19.0,419.0,29.0,Latin America and the Caribbean,Caribbean,AI,Anguilla *,Anguilla,{3B1FFACC-7CBC-48A8-870D-3D3A22774221},3,GBR,{8DE43565-A675-4EEA-AF35-165EEFA5B24D},"POLYGON ((-62.98077 18.24796, -62.98051 18.246..."
4,5,ALA,248,Åland Islands,Åland Islands (Finland),EUR,150.0,154.0,NaN,Northern Europe,None,AX,Îles Åland (Finlande),Îles d’Åland,{FFD8BC5F-2088-4D00-ADDC-FA7B591050CA},4,FIN,{9D3F0CE2-CAC7-46D9-9E05-794F88DE5858},"MULTIPOLYGON (((19.92624 60.42022, 19.93433 60..."


In [14]:
# subareas like the ones below need to be combined
df_bnda.query('iso3cd in ("PRT", "USA", "ESP")').sort_values("iso3cd")

,objectid,iso3cd,m49_cd,nam_en,lbl_en,georeg,geo_cd,sub_cd,int_cd,subreg,intreg,iso2cd,lbl_fr,name_fr,globalid,stscod,admiso,globalid_1,geometry
253,267,ESP,724,Spain,SPAIN,EUR,150.0,39.0,NaN,Southern Europe,None,ES,ESPAGNE,Espagne,{72455610-029D-4D78-A567-FF858EB52897},1,ESP,{2809CF48-D821-49F3-A7D4-42108BDD7603},"MULTIPOLYGON (((-1.78598 43.35048, -1.63 43.28..."
254,268,ESP,724,Canary Islands,Canary Islands (Sp.),EUR,150.0,39.0,NaN,Southern Europe,None,ES,Îles Canaries (Esp.),Îles Canaries (partie espagne),{72455610-029D-4D78-A567-FF858EB52897},4,ESP,{58508103-2269-4B94-846E-5740623947E3},"MULTIPOLYGON (((-16.55163 28.02303, -16.64587 ..."
257,273,PRT,620,Portugal,PORTUGAL,EUR,150.0,39.0,NaN,Southern Europe,None,PT,PORTUGAL,Portugal,{D959E8DC-4E7F-451D-A21D-1FF004FB54CC},1,PRT,{E9AE7A6A-DB54-4E49-B263-82E82CC6F3FC},"POLYGON ((-7.32266 38.47867, -7.13122 38.23052..."
258,274,PRT,620,Madeira Island,Madeira Islands (Port.),EUR,150.0,39.0,NaN,Southern Europe,None,PT,Île de Madère (Port.),Île de Madère (partie portugal),{D959E8DC-4E7F-451D-A21D-1FF004FB54CC},4,PRT,{F4D4E098-60D7-48BE-88D6-8204CBD58C7C},"MULTIPOLYGON (((-16.85417 32.63474, -16.94626 ..."
259,277,PRT,620,Azores Islands,Azores Islands (Port.),EUR,150.0,39.0,NaN,Southern Europe,None,PT,Îles des Açores (Port.),Îles des Açores (partie portugal),{D959E8DC-4E7F-451D-A21D-1FF004FB54CC},4,PRT,{07EB908B-50CE-4858-A167-3D7E6A0F9A09},"MULTIPOLYGON (((-25.581 37.81297, -25.56622 37..."
228,230,USA,840,United States of America,Hawaii (USA),AME,19.0,21.0,NaN,Northern America,None,US,Hawaï (États-Unis),États-Unis d’Amérique,{14698C55-70B6-47B7-9533-BDDB5A73A69B},4,USA,{8CCFC775-549D-4CEE-847D-5780BB2FC868},"MULTIPOLYGON (((-154.96941 19.35403, -155.5687..."
261,283,USA,840,United States of America,UNITED STATES OF AMERICA,AME,19.0,21.0,NaN,Northern America,None,US,ÉTATS-UNIS D’AMÉRIQUE,États-Unis d’Amérique,{14698C55-70B6-47B7-9533-BDDB5A73A69B},1,USA,{63101F20-6E1F-40F4-9EBA-315BB0228510},"MULTIPOLYGON (((-67.09448 45.07453, -66.97003 ..."


In [15]:
# get the names of the countries (not areas)
mapping = dict(
    df_bnda.sort_values("stscod", ascending=True)
    .drop_duplicates(["iso3cd"], keep="first")[["iso3cd", "nam_en"]]
    .values
)
assert mapping["PRT"] == "Portugal"
assert mapping["USA"] == "United States of America"

In [16]:
# merge areas within countries proper
df_bnda = df_bnda.reindex(columns=["iso3cd", "geometry"])
df_bnda = df_bnda.dissolve(by="iso3cd", as_index=False)
df_bnda["nam_en"] = df_bnda["iso3cd"].map(mapping)
print("Shape:", df_bnda.shape)
display(df_bnda.head())

Shape: (250, 3)


,iso3cd,geometry,nam_en
0,ABW,"POLYGON ((-70.05101 12.62242, -70.04847 12.621...",Aruba
1,AFG,"POLYGON ((74.88986 37.23409, 74.69366 37.2662,...",Afghanistan
2,AGO,"MULTIPOLYGON (((24.03344 -11.19853, 23.98423 -...",Angola
3,AIA,"POLYGON ((-62.98077 18.24796, -62.98051 18.246...",Anguilla
4,ALA,"MULTIPOLYGON (((20.4481 60.00844, 20.44952 60....",Åland Islands


In [ ]:
# df_bnda.to_file("src/data/countries.geojson", driver="GeoJSON")

### Data Preparation

- Extract texts from files.
- Chunk and embed the texts.
- Assign custom categories for the texts.
- Ingest data into the database.

In [18]:
df_registry = pd.read_csv(PATH_DATA.joinpath(f"ndc-registry-v{VERSION}.csv"))
print("Shape:", df_registry.shape)
display(df_registry.head())

Shape: (233, 8)


,iso,party,version,date,type,title,url,file_name
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf
1,MDA,Republic of Moldova,3,2025-05-06,original,Republic of Moldova NDC 3.0,https://unfccc.int/sites/default/files/2025-05...,mda-original-25-05-06.pdf
2,KEN,Kenya,0,2025-04-30,original,Kenya's Second Nationally Determined Contribut...,https://unfccc.int/sites/default/files/2025-05...,ken-original-25-04-30.pdf
3,ZMB,Zambia,0,2025-03-10,original,Provisional NDC 3.0,https://unfccc.int/sites/default/files/2025-03...,zmb-original-25-03-10.pdf
4,CUB,Cuba,0,2025-02-27,original,Cuba NDC 3.0,https://unfccc.int/sites/default/files/2025-02...,cub-original-25-02-27.pdf


#### Text Extraction

Use this section to extract text from documents and identify its main language. The code makes use of an internal UNDP API to extract text. You can modify `extract_text` function at the beginning of the notebook to use any other open-source or third-party solution to perfom the extraction as long as the function signature is unchanged.

In [19]:
# init `pages` column to be able to assign an object with `at`
df_registry["pages"] = None
with requests.Session() as session:
    for index, row in tqdm(df_registry.iterrows(), total=len(df_registry)):
        file_path = CORPUS_DATA.joinpath(row.file_name)
        try:
            language, pages = extract_text(file_path, session)
        except Exception as e:
            # check the errors
            print(row.file_name, e)
            language, pages = None, None
        finally:
            df_registry.at[index, "language"] = language
            df_registry.at[index, "pages"] = pages

print("Shape:", df_registry.shape)
display(df_registry.head())

100%|██████████| 233/233 [05:10<00:00,  1.33s/it]

Shape: (233, 10)


,iso,party,version,date,type,title,url,file_name,pages,language
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,[index=0 text='Nationally Determined Contribut...,eng
1,MDA,Republic of Moldova,3,2025-05-06,original,Republic of Moldova NDC 3.0,https://unfccc.int/sites/default/files/2025-05...,mda-original-25-05-06.pdf,[index=0 text='1 | P a g e Government of the R...,eng
2,KEN,Kenya,0,2025-04-30,original,Kenya's Second Nationally Determined Contribut...,https://unfccc.int/sites/default/files/2025-05...,ken-original-25-04-30.pdf,[index=0 text='Kenya’s Second Nationally Deter...,eng
3,ZMB,Zambia,0,2025-03-10,original,Provisional NDC 3.0,https://unfccc.int/sites/default/files/2025-03...,zmb-original-25-03-10.pdf,"[index=0 text=""REPUBLIC OF ZAMBIA 1.0 Backgrou...",eng
4,CUB,Cuba,0,2025-02-27,original,Cuba NDC 3.0,https://unfccc.int/sites/default/files/2025-02...,cub-original-25-02-27.pdf,[index=0 text='República de Cuba Febrero 2025...,spa


In [20]:
df_registry["language"].value_counts(dropna=False)

language
eng    174
spa     24
fra     24
rus      6
ara      4
zho      1
Name: count, dtype: int64

In [21]:
# no text extracted from the following NDCs
mask = df_registry["pages"].apply(get_pages_length).lt(30)
df_registry.loc[mask]

,iso,party,version,date,type,title,url,file_name,pages,language
14,LSO,Lesotho,0,2025-02-05,original,Lesotho Second NDC,https://unfccc.int/sites/default/files/2025-02...,lso-original-25-02-05.pdf,"[index=0 text='', index=1 text='', index=2 tex...",eng
63,KIR,Kiribati,2,2023-03-02,original,Kiribati Enhanced NDC,https://unfccc.int/sites/default/files/NDC/202...,kir-original-23-03-02.pdf,"[index=0 text='', index=1 text='', index=2 tex...",eng
123,MLI,Mali,2,2021-10-11,original,Mali First NDC (Updated submission),https://unfccc.int/sites/default/files/NDC/202...,mli-original-21-10-11.pdf,"[index=0 text='', index=1 text='', index=2 tex...",fra
147,NGA,Nigeria,3,2021-07-30,original,Nigeria First NDC (Updated submission),https://unfccc.int/sites/default/files/NDC/202...,nga-original-21-07-30.pdf,"[index=0 text='', index=1 text=' ...",eng
179,MHL,Marshall Islands,3,2020-12-31,original,Marshall Islands Second NDC (Updated submission),https://unfccc.int/sites/default/files/NDC/202...,mhl-original-20-12-31.pdf,"[index=0 text='', index=1 text='', index=2 tex...",eng
186,KEN,Kenya,2,2020-12-28,original,Kenya First NDC (Updated submission),https://unfccc.int/sites/default/files/NDC/202...,ken-original-20-12-28.pdf,"[index=0 text='', index=1 text='', index=2 tex...",eng
201,PRK,Democratic People's Republic of Korea,2,2019-09-19,original,Democratic People's Republic of Korea First ND...,https://unfccc.int/sites/default/files/NDC/202...,prk-original-19-09-19.pdf,"[index=0 text='', index=1 text='', index=2 tex...",eng


#### Document Preprocessing

Use this section to filter out malformed texts and chunking pages for RAG.

In [22]:
# subset data in supported languages where extracted text is present, i.e., at least 30 tokens on average
mapping = {
    "eng": "en",
    "spa": "es",
    "fra": "fr",
    "rus": "ru",
    "ara": "ar",
    "zho": "zh",
}
mask = df_registry["language"].isin(mapping) & df_registry["pages"].apply(
    get_pages_length
).ge(30)
df_registry["chunks"] = df_registry["pages"].apply(chunk_pages)
df_registry.drop(["pages"], axis=1, inplace=True)

# "explode" so that each row corresponds to a single chunk with metadata
df_chunks = df_registry.loc[mask].explode("chunks", ignore_index=True)
df_chunks = df_chunks.join(pd.DataFrame(df_chunks["chunks"].tolist()))
df_chunks.drop(["chunks"], axis=1, inplace=True)

# convert date format and standardise language names to ISO 639 Set 1
df_chunks["date"] = pd.to_datetime(df_chunks["date"])
df_chunks["language"] = df_chunks["language"].map(mapping)

# remove very short and/or corrupted chunks
mask = df_chunks["text"].apply(get_text_length).ge(32)
df_chunks = df_chunks.loc[mask].copy()
df_chunks.reset_index(drop=True, inplace=True)

print("Shape:", df_chunks.shape)
display(df_chunks.head())

Shape: (39187, 11)


,iso,party,version,date,type,title,url,file_name,language,text,pages
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,Nationally Determined Contribution (NDC) 3.0 G...,"[0, 1, 2]"
1,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,"21 paragraph 23 and 24, and other relevant pro...",[2]
2,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,full implementation. NDC 3.0 is fair and ambit...,[2]
3,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,-zero Emissions (LTS) (2021). The provision in...,[2]
4,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,to avoid the residual risks of climate change ...,"[2, 3]"


In [23]:
df_chunks["language"].value_counts(dropna=False)

language
en    21145
es    10876
fr     5630
ru      920
ar      361
zh      255
Name: count, dtype: int64

In [24]:
df_chunks["text"].apply(get_text_length).describe().round(2)

count    39187.00
mean       195.27
std         29.70
min         32.00
25%        184.00
50%        201.00
75%        215.00
max        255.00
Name: text, dtype: float64

#### Category Assignment

Assign custom categories to chunks using an expert-curated list of multi-language regex expressions.

In [25]:
df_categories = pd.read_csv(PATH_DATA.joinpath("energy-categories.csv"))
print("Shape:", df_categories.shape)
display(df_categories.head())

Shape: (59, 9)


,category,keyword_en,regex_en,keyword_fr,regex_fr,keyword_es,regex_es,keyword_ru,regex_ru
0,Energy access,grids,\bgrid(s)?\b,réseaux,\bréseau(x|x)?\b,grids,\bgrid(s)?\b,сети,\bсет(ь|и|ей|ям|ями|ях)?\b
1,Energy access,electricity access,\belectricity access\b,accès à l'électricité,\baccès à l'électricité\b,acceso a la electricidad,\bacceso a la electricidad\b,доступ к электричеству,\bдоступ к электричеству\b
2,Energy access,cook,\bcook(ing)?\b,cuire,\bcuir(e|es|ons|ez|ent)?\b,cocinar,\bcocin(ar|ando|ado|ada|ados|adas)?\b,готовить,\bготов(ить|лю|ишь|ит|им|ите|ят)\b
3,Energy access,stove,\bstove(s)?\b,poêle,\bpoêle(s)?\b,estufa(s)?,\bestufa(s)?\b,печь,\bпеч(ь|и|ей|ям|ями|ях)?\b
4,Energy access,energy access,\benergy access\b,accès à l'énergie,\baccès à l'énergie\b,acceso a la energía,\bacceso a la energía\b,доступ к энергии,\bдоступ к энергии\b


In [26]:
# this only assigns categories to chunks in supported languages
df_chunks["categories"] = [set() for _ in range(len(df_chunks))]
for row in tqdm(df_categories.to_dict(orient="records")):
    for language in ("en", "fr", "es", "ru"):
        mask = df_chunks["language"].eq(language)
        pattern = row[f"regex_{language}"]
        mask &= df_chunks.loc[mask, "text"].str.contains(
            pattern, case=False, regex=True
        )
        df_chunks.loc[mask, "categories"].apply(lambda x: x.add(row["category"]))
df_chunks["categories"] = df_chunks["categories"].apply(
    lambda x: sorted(x) if x else None
)
print("Shape:", df_chunks.shape)
display(df_chunks.head())

  0%|          | 0/59 [00:00<?, ?it/s]/var/folders/1b/hls62h1s7l91l0tyg90qskcc0000gn/T/ipykernel_25819/2487745529.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask &= df_chunks.loc[mask, "text"].str.contains(
/var/folders/1b/hls62h1s7l91l0tyg90qskcc0000gn/T/ipykernel_25819/2487745529.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask &= df_chunks.loc[mask, "text"].str.contains(
/var/folders/1b/hls62h1s7l91l0tyg90qskcc0000gn/T/ipykernel_25819/2487745529.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask &= df_chunks.loc[mask, "text"].str.contains(
/var/folders/1b/hls62h1s7l91l0tyg90qskcc0000gn/T/ipykernel_25819/2487745529.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match

Shape: (39187, 12)


,iso,party,version,date,type,title,url,file_name,language,text,pages,categories
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,Nationally Determined Contribution (NDC) 3.0 G...,"[0, 1, 2]",[Energy efficiency]
1,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,"21 paragraph 23 and 24, and other relevant pro...",[2],[Energy efficiency]
2,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,full implementation. NDC 3.0 is fair and ambit...,[2],None
3,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,-zero Emissions (LTS) (2021). The provision in...,[2],"[Energy efficiency, Energy transition]"
4,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,to avoid the residual risks of climate change ...,"[2, 3]","[Energy efficiency, Energy resilience, Energy ..."


In [27]:
df_chunks["categories"].explode().value_counts(dropna=False)

categories
None                 20371
Energy transition    12953
Energy efficiency    11609
Renewable energy      2993
Energy access         1196
Energy resilience      383
Name: count, dtype: int64

#### Text Embedding

Use this section to embed text for vector search using a text embedding model.

In [28]:
tokens = df_chunks["text"].apply(get_text_length).sum() / 1000
price = 0.0001  # per 1k tokens
print(f"Approximate cost: ${tokens * price:.2f}")

Approximate cost: $0.77


In [29]:
batch_size = 16
embeddings = []
df_chunks.reset_index(drop=True, inplace=True)
for i in tqdm(range(0, len(df_chunks), batch_size)):
    texts = df_chunks.loc[i : i + batch_size - 1, "text"].tolist()
    try:
        embeddings.extend(embed_texts(texts))
    except Exception as e:
        print(e)
        embeddings.extend([None] * len(texts))
df_chunks["vector"] = embeddings
print("Shape:", df_chunks.shape)
display(df_chunks.head())

100%|██████████| 2450/2450 [1:53:26<00:00,  2.78s/it]  

Shape: (39187, 13)


,iso,party,version,date,type,title,url,file_name,language,text,pages,categories,vector
0,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,Nationally Determined Contribution (NDC) 3.0 G...,"[0, 1, 2]",[Energy efficiency],"[-0.002757846610620618, -0.010120416060090065,..."
1,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,"21 paragraph 23 and 24, and other relevant pro...",[2],[Energy efficiency],"[0.0004244866140652448, -0.0072736130096018314..."
2,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,full implementation. NDC 3.0 is fair and ambit...,[2],None,"[-0.002929023467004299, -0.011742690578103065,..."
3,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,-zero Emissions (LTS) (2021). The provision in...,[2],"[Energy efficiency, Energy transition]","[0.0009831269271671772, -0.008368322625756264,..."
4,NPL,Nepal,3,2025-05-19,original,Nationally Determined Contribution (NDC) 3.0,https://unfccc.int/sites/default/files/2025-05...,npl-original-25-05-19.pdf,en,to avoid the residual risks of climate change ...,"[2, 3]","[Energy efficiency, Energy resilience, Energy ...","[-1.1862841347465292e-05, -0.00444221124053001..."


In [ ]:
df_chunks.to_json(
    PATH_DATA.joinpath(f"ndc-chunks-v{VERSION}.jsonl"),
    orient="records",
    lines=True,
)

#### Data Ingestion

Load the data into a LanceDB instance connected to the app.

In [ ]:
# overwrite the data
load_data(df_chunks)

39187